In [3]:
from datasets import load_dataset
folio = load_dataset("yale-nlp/FOLIO")
#p_folio = load_dataset("yale-nlp/P-FOLIO")

In [6]:
folio_train = folio['train']
folio_train

Dataset({
    features: ['story_id', 'premises', 'premises-FOL', 'conclusion', 'conclusion-FOL', 'label', 'example_id'],
    num_rows: 1001
})

In [14]:
clean_premises = []
clean_labels = []
for i in range(len(folio_train['label'])):
    if folio_train['label'][i] == 'True':
        clean_labels.append(1)
        clean_premises.append(folio_train['premises'][i])
    elif folio_train['label'][i] == 'False':
        clean_labels.append(0)
        clean_premises.append(folio_train['premises'][i])

print(len(clean_premises), len(clean_labels), len(folio_train['label']))

677 677 1001


**Notas**

1. Un primer acercamiento es simplificar el dataset ignorando los valores 'Uncertain'
2. Tal vez se puede medir algún nivel de "relevancia" para decantar el etiquetado final.

In [ ]:
import torch
from transformers import GPT2Tokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained('openai-community/gpt2')


In [ ]:
from trl import SFTConfig, SFTTrainer

training_args = SFTConfig(
    learning_rate = 1e-05,
    weight_decay = 0.02,
    num_train_epochs = 2.0
)

trainer = SFTTrainer(
    model = model,
    train_dataset = 
)